In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

customers = pd.read_csv("customers_data.csv")
products = pd.read_csv("products_data.csv")
transactions = pd.read_csv("transactions_data.csv")

#View shape (rows, columns)
print(customers.shape)
print(products.shape)
print(transactions.shape)



FileNotFoundError: [Errno 2] No such file or directory: 'customers_data.csv'

In [ ]:
#Preview Data

customers.head(10)
products.head(10)
transactions.head(10)

We'll start by merging the cleaned transactions data with customers using Company_ID. Then, we'll take that combined data and merge it with the products data using Product_ID. This will give you a comprehensive dataset that includes information from all three sources, with all transactions preserved

In [ ]:
# Merge transactions_cleaned with customers on Company_ID
merged_data = pd.merge(transactions_cleaned, customers, on='Company_ID', how='left')

# Merge the result with products on Product_ID
merged_data = pd.merge(merged_data, products, on='Product_ID', how='left')

# Display the first few rows of the merged DataFrame
display(merged_data.head())

# Display the shape of the merged DataFrame
print(f"Shape of the merged DataFrame: {merged_data.shape}")

In [ ]:
products.head(10)

In [ ]:
#Duplicate columns = columns with identical values.
def duplicate_columns_info(df):
    dup_mask = df.T.duplicated()
    dup_columns = df.columns[dup_mask]

    return {
        "duplicate_columns": list(dup_columns),
        "duplicate_count": dup_mask.sum()
    }

#Check duplicate columns for each file
customers_dups = duplicate_columns_info(customers)
transactions_dups = duplicate_columns_info(transactions)
products_dups = duplicate_columns_info(products)

customers_dups, transactions_dups, products_dups

def print_duplicate_columns(name, result):
    print(f"\n{name}")
    print("Duplicate column count:", result["duplicate_count"])
    print("Duplicate columns:", result["duplicate_columns"])

print_duplicate_columns("Customers Data", customers_dups)
print_duplicate_columns("Transactions Data", transactions_dups)
print_duplicate_columns("Products Data", products_dups)


In [ ]:
#Drop Unnamed Column as this is data is too redundant with Transaction_ID

transactions.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')

In [ ]:
customers.head(10)
products.head(10)
transactions.head(10)

,unnamed:_0,transaction_id,company_id,product_id,quantity,transaction_date,product_price,total_cost,high_value_transaction
0,0.0,1.0,88.0,6.0,11.0,2024-03-26,194379.147964,1075200.0,0
1,1.0,2.0,29.0,19.0,16.0,NaN,97930.993380,1428000.0,0
2,2.0,5005.0,28.0,18.0,6.0,NaN,126095.547778,940800.0,0
3,3.0,4.0,85.0,12.0,12.0,NaN,131297.783516,1008000.0,0
4,4.0,5.0,47.0,3.0,8.0,NaN,99575.609634,705600.0,0
5,5.0,6.0,80.0,11.0,4.0,2021-07-12,160658.675350,627200.0,0
6,4997.5,7.0,50.0,10.0,14.0,2023-11-01,133548.749710,1960000.0,1
7,7.0,8.0,21.0,20.0,7.0,NaN,229217.941468,1792000.0,0
8,8.0,9.0,96.0,1.0,14.0,NaN,144758.783254,1344000.0,0
9,9.0,10.0,3.0,20.0,6.0,NaN,238293.851303,1120000.0,0


In [ ]:
#Standardize the Transaction Date format to better understand column

transactions["Transaction_Date"] = pd.to_datetime(transactions["Transaction_Date"], errors='coerce')

transactions.head()

KeyError: 'Transaction_Date'

### Imputing Missing `Transaction_Date` values with the Mode

Given the high percentage of `NaT` values in `Transaction_Date` (4329 out of 9000 rows in `transactions_cleaned`), dropping these rows would result in substantial data loss. A common strategy to handle such a situation is to impute the missing values with the **mode** (most frequent value) of the column. This approach helps preserve the dataset size while assigning a statistically representative date.

First, let's find the most frequent `Transaction_Date`.

In [ ]:
# Calculate the mode of the 'Transaction_Date' column in transactions_cleaned
# .mode()[0] is used because mode() can return multiple modes if they have the same frequency
most_frequent_date = transactions_cleaned['Transaction_Date'].mode()[0]

print(f"The most frequent transaction date is: {most_frequent_date}")

Now, we will fill the `NaT` values in the `Transaction_Date` column with this most frequent date.

In [ ]:
# Fill NaT values in 'Transaction_Date' with the most frequent date
transactions_cleaned['Transaction_Date'].fillna(most_frequent_date, inplace=True)

# Verify that there are no more NaT values in 'Transaction_Date'
print("Missing values in 'Transaction_Date' after imputation:")
display(transactions_cleaned['Transaction_Date'].isna().sum())

# Display the first few rows of the updated DataFrame to see the changes
display(transactions_cleaned.head())

In [ ]:
# Display summary statistics for numerical columns in merged_data
display(merged_data.describe())

In [ ]:
#Count missing values per column

transactions.isna().sum()

In [ ]:
# Save the merged DataFrame to a CSV file
merged_data.to_csv('merged_data.csv', index=False)

print("merged_data.csv has been successfully created and saved.")

In [ ]:
print("Missing values in 'Transaction_Date' column of merged_data:")
display(merged_data['Transaction_Date'].isna().sum())

# Task
Analyze the `merged_data` DataFrame by checking data types, visualizing missing values and distributions of numerical columns, inspecting categorical columns for inconsistencies, and identifying redundant or irrelevant columns, especially `Product_Price_x` and `Product_Price_y`.

## Check Data Types

### Subtask:
Examine the data types of all columns in `merged_data` using `.info()` to identify any columns with incorrect or mixed data types.


**Reasoning**:
To examine the data types of all columns in `merged_data` and identify any columns with incorrect or mixed data types, I will use the `.info()` method on the `merged_data` DataFrame.



In [ ]:
merged_data.info()

## Analyze Missing Values

### Subtask:
Identify and quantify missing values across all columns in `merged_data` using `.isna().sum()` and visualize them to understand their distribution.


**Reasoning**:
To visualize the distribution of missing values, I will create a heatmap using seaborn.heatmap() on the boolean DataFrame indicating missing values in `merged_data`.



In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(merged_data.isna(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap in merged_data')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()

**Reasoning**:
To quantify the missing values, I will use `.isna().sum()` on the `merged_data` DataFrame, which will display the count of missing values for each column.



In [ ]:
print("Missing values per column in merged_data:")
display(merged_data.isna().sum())

## Inspect Numerical Columns for Outliers and Distribution

### Subtask:
Generate descriptive statistics and visualize the distributions of numerical columns in `merged_data` to detect outliers and understand their spread.


**Reasoning**:
To display descriptive statistics for all numerical columns in `merged_data`, I will use the `.describe()` method as specified in the instructions. This will provide a summary of central tendency, dispersion, and shape of the distribution of each numerical column.



In [ ]:
print("Descriptive statistics for numerical columns in merged_data:")
display(merged_data.describe())

**Reasoning**:
To visualize the distributions of numerical columns and detect outliers, I will create histograms and box plots for the specified numerical columns: Transaction_ID, Company_ID, Product_ID, Quantity, Product_Price_x, Total_Cost, and Company_Profit. Histograms will show the shape of the distributions, while box plots will highlight the spread and potential outliers.



In [ ]:
numerical_cols = ['Transaction_ID', 'Company_ID', 'Product_ID', 'Quantity', 'Product_Price_x', 'Total_Cost', 'Company_Profit']

# Create histograms for numerical columns
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols):
    plt.subplot(3, 3, i + 1) # Arrange plots in a 3x3 grid
    sns.histplot(merged_data[col].dropna(), kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# Create box plots for numerical columns
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols):
    plt.subplot(3, 3, i + 1) # Arrange plots in a 3x3 grid
    sns.boxplot(y=merged_data[col].dropna())
    plt.title(f'Box Plot of {col}')
    plt.ylabel(col)
plt.tight_layout()
plt.show()

## Inspect Categorical Columns for Inconsistencies

### Subtask:
Identify categorical columns in `merged_data` and examine their unique values and value counts to detect inconsistencies, typos, or unexpected entries.

**Reasoning**:
To identify categorical columns and prepare for inspection, I will first get a list of columns with 'object' data type from the `merged_data` DataFrame.



In [ ]:
categorical_cols = merged_data.select_dtypes(include='object').columns
print("Categorical columns identified:")
print(categorical_cols)

**Reasoning**:
To detect inconsistencies, typos, or unexpected entries, I will iterate through each identified categorical column and display its unique values and their counts.



In [ ]:
for col in categorical_cols:
    print(f"\nUnique values and their counts for column '{col}':")
    display(merged_data[col].value_counts(dropna=False))


## Identify Redundant or Irrelevant Columns

### Subtask:
Identify and address redundant or irrelevant columns, specifically focusing on `Product_Price_x` and `Product_Price_y` in `merged_data`.

## Identify Redundant or Irrelevant Columns

### Subtask:
Identify and address redundant or irrelevant columns, specifically focusing on `Product_Price_x` and `Product_Price_y` in `merged_data`.

**Reasoning**:
To better understand the relationship and potential redundancy between `Product_Price_x` and `Product_Price_y`, I will display the first few rows of the `merged_data` DataFrame focusing on these two columns along with `Product_ID` and `Product_Name`.



In [ ]:
print("Comparing 'Product_Price_x' and 'Product_Price_y':")
display(merged_data[['Product_ID', 'Product_Name', 'Product_Price_x', 'Product_Price_y']].head(10))

**Reasoning**:
To make `Product_Price_y` comparable with `Product_Price_x` and usable for numerical analysis, I need to clean it by removing non-numeric characters (like '?' and ',') and then convert its data type to a float. This will standardize the price format across both columns.



In [ ]:
#Cleaned Product_Price_y by cleaning and removing on-numeric characters (like '?' and ',') and then convert its data type to a float. This will standardize the price format across both columns.

merged_data['Product_Price_y_cleaned'] = merged_data['Product_Price_y'].astype(str).str.replace('?', '', regex=False).str.replace(',', '', regex=False)
merged_data['Product_Price_y_cleaned'] = pd.to_numeric(merged_data['Product_Price_y_cleaned'], errors='coerce')

print("Data types after cleaning Product_Price_y:")
display(merged_data[['Product_Price_x', 'Product_Price_y', 'Product_Price_y_cleaned']].dtypes)
print("First 10 rows after cleaning Product_Price_y:")
display(merged_data[['Product_ID', 'Product_Name', 'Product_Price_x', 'Product_Price_y', 'Product_Price_y_cleaned']].head(10))

**Reasoning**:
To quantify the relationship and potential redundancy between `Product_Price_x` and the newly cleaned `Product_Price_y_cleaned`, I will calculate their correlation coefficient. A high correlation would suggest they represent the same information, making one of them redundant.



In [ ]:
#Check correlation between Product_Price_x and Product_Price_y_cleaned

correlation = merged_data['Product_Price_x'].corr(merged_data['Product_Price_y_cleaned'])
print(f"Correlation between Product_Price_x and Product_Price_y_cleaned: {correlation}")

# Also, visually compare them with a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Product_Price_x', y='Product_Price_y_cleaned', data=merged_data)
plt.title('Scatter Plot of Product_Price_x vs. Product_Price_y_cleaned')
plt.xlabel('Product_Price_x')
plt.ylabel('Product_Price_y_cleaned')
plt.grid(True)
plt.show()

**Reasoning**:
Since the correlation between 'Product_Price_x' and 'Product_Price_y_cleaned' is very high (0.979), indicating strong redundancy, I will drop the 'Product_Price_y' and 'Product_Price_y_cleaned' columns, keeping 'Product_Price_x' as the primary price column.



In [ ]:


merged_data.drop(columns=['Product_Price_y', 'Product_Price_y_cleaned'], inplace=True, errors='ignore')
print("Dropped 'Product_Price_y' and 'Product_Price_y_cleaned' columns (if they existed).")
print("Remaining columns:")
display(merged_data.columns)

## Summary:

### Data Analysis Key Findings

*   **Data Types and Potential Issues**:
    *   `Transaction_Date` is currently an `object` type and requires conversion to a datetime format for time-series analysis.
    *   `Product_Price_y` was initially an `object` type, indicating non-numeric characters or string storage, necessitating cleaning and conversion to a numeric type. `Product_Price_x` is already a `float64`.
*   **Missing Values**:
    *   `Company_Profit` has the highest number of missing values with 1148.
    *   `Company_Name` and `Address` both have 536 missing values.
    *   `Product_Name` and `Product_Price_y` both have 530 missing values.
    *   `Transaction_ID`, `Company_ID`, `Product_ID`, `Quantity`, `Transaction_Date`, `Product_Price_x`, and `Total_Cost` have no missing values.
*   **Numerical Column Distributions**:
    *   `Company_Profit` has a lower count of 4181 non-null entries out of 5329 total, confirming the presence of missing values.
    *   The `Quantity` column includes a minimum value of 0, which may represent specific business cases (e.g., returns or failed transactions) or data entry errors.
    *   `Product_Price_x` and `Total_Cost` show large standard deviations, indicating a wide spread in their values.
    *   Box plots revealed potential outliers in `Transaction_ID`, `Product_Price_x`, `Total_Cost`, and `Company_Profit`.
*   **Redundant Columns (`Product_Price_x` vs. `Product_Price_y`)**:
    *   `Product_Price_y` contained non-numeric characters such as '?' and ',', making it inconsistent with `Product_Price_x`.
    *   After cleaning `Product_Price_y` into a new numeric column `Product_Price_y_cleaned`, its correlation with `Product_Price_x` was assessed.
    *   Ultimately, the original `Product_Price_y` and the cleaned `Product_Price_y_cleaned` columns were dropped, implying `Product_Price_x` was selected as the primary and more reliable product price column.

### Insights or Next Steps

*   Convert the `Transaction_Date` column to a datetime format to enable proper time-series analysis and calculations.
*   Develop a strategy to handle the significant missing values, particularly in `Company_Profit`, `Company_Name`, `Address`, and `Product_Name`, considering imputation, deletion, or further investigation into the cause of missingness.
*   Investigate the minimum `Quantity` of 0 to understand its business implications and decide whether these entries should be filtered, adjusted, or kept based on context.


# Task
Convert 'Transaction_Date' to a datetime format in the `merged_data` DataFrame, then address missing values in 'Company_Profit', 'Company_Name', 'Address', and 'Product_Name' in `merged_data`. Finally, investigate entries where 'Quantity' is 0 in `merged_data`.

## Date Type Conversion

### Subtask:
Convert 'Transaction_Date' to a datetime format to enable time-series analysis and feature extraction.


**Reasoning**:
To convert the 'Transaction_Date' column to a datetime format and handle potential errors, I will use `pd.to_datetime` with `errors='coerce'`. After the conversion, I will verify the data type using `merged_data.info()` to ensure the change was successful.



In [ ]:
merged_data['Transaction_Date'] = pd.to_datetime(merged_data['Transaction_Date'], errors='coerce')

print("Data type after converting 'Transaction_Date' to datetime:")
merged_data.info()

## Handle Missing Values

### Subtask:
Address missing values in 'Company_Profit', 'Company_Name', 'Address', and 'Product_Name' in `merged_data`.

#### Reasoning and Strategy:

*   **Company_Profit (Numerical):** Given its numerical nature, a suitable imputation strategy would be to fill missing values with the **median**. The median is less sensitive to outliers compared to the mean, making it a robust choice for financial data like profit.

*   **Company_Name (Categorical/Object):** Missing company names could indicate unknown entities or data entry issues. Imputing with a placeholder like **'Unknown'** is a common and safe approach for categorical data, as it preserves the information that the original name was missing without introducing false relationships.

*   **Address (Categorical/Object):** Similar to `Company_Name`, missing addresses can be imputed with **'Unknown'** to explicitly mark the absence of this information, preventing potential geographical or logistical biases in analysis.

*   **Product_Name (Categorical/Object):** For missing product names, imputing with **'Unknown Product'** allows us to retain these transaction records while acknowledging that the specific product information is unavailable. This prevents the loss of potentially valuable transaction data.

**Note:** Before imputation, I will re-check the missing values count to ensure I am targeting the correct columns and have the latest information.

**Reasoning**:
Before imputing missing values, I will re-check the current count of missing values for each column in `merged_data` to ensure the strategy is based on the most up-to-date information.



In [ ]:
#Check Missing Values per column

print("Missing values per column in merged_data before imputation:")
display(merged_data.isna().sum())

**Reasoning**:
To address the `FutureWarning` related to `inplace=True` when using `fillna()`, I will rewrite the code to explicitly reassign the filled columns, which is a more robust and recommended practice in pandas.



In [ ]:
median_company_profit = merged_data['Company_Profit'].median()
merged_data['Company_Profit'] = merged_data['Company_Profit'].fillna(median_company_profit)

merged_data['Company_Name'] = merged_data['Company_Name'].fillna('Unknown')
merged_data['Address'] = merged_data['Address'].fillna('Unknown')
merged_data['Product_Name'] = merged_data['Product_Name'].fillna('Unknown Product')

print("Missing values per column in merged_data after imputation:")
display(merged_data.isna().sum())

## Investigate Quantity = 0

### Subtask:
Investigate entries in `merged_data` where the 'Quantity' column has a value of 0 to understand their implications and decide on appropriate handling.

#### Reasoning and Strategy:

A minimum `Quantity` of 0 could represent various scenarios such as cancelled orders, returns, or data entry errors. It's crucial to understand the proportion and context of these entries to determine the best course of action. Strategies could include:

*   **Filtering out**: If these entries genuinely represent non-sales events, removing them might be appropriate for sales analysis.
*   **Further investigation**: If they represent returns or specific business cases, they might need to be treated differently (e.g., recorded as negative quantities or analyzed separately).

First, I will quantify how many records have a `Quantity` of 0 and examine their characteristics.

**Reasoning**:
To quantify the records with a 'Quantity' of 0, I will filter the `merged_data` DataFrame for these entries, calculate their count, and display the first few rows to understand their characteristics.



In [ ]:
quantity_zero_entries = merged_data[merged_data['Quantity'] == 0]

print(f"Number of records where Quantity is 0: {len(quantity_zero_entries)}")
print("First 10 records where Quantity is 0:")
display(quantity_zero_entries.head(10))


## Feature Engineering: Time-Based Features

### Subtask:
Extract features such as year, month, day of week, and possibly quarter from the 'Transaction_Date' to capture temporal patterns.


**Reasoning**:
To extract time-based features from the 'Transaction_Date' column, I will create new columns for the year, month, day of the week, and quarter using pandas' datetime properties, and then display the head of the DataFrame to verify the new columns.



In [ ]:
merged_data['Transaction_Year'] = merged_data['Transaction_Date'].dt.year
merged_data['Transaction_Month'] = merged_data['Transaction_Date'].dt.month
merged_data['Transaction_DayOfWeek'] = merged_data['Transaction_Date'].dt.dayofweek
merged_data['Transaction_Quarter'] = merged_data['Transaction_Date'].dt.quarter

print("First 10 rows of merged_data with new time-based features:")
display(merged_data.head(10))

## Feature Engineering: Customer-Centric Aggregates

### Subtask:
Create features like total number of transactions per customer, total quantity purchased, total spending, and average transaction value per customer.


**Reasoning**:
To create customer-centric aggregate features, I will group the `merged_data` DataFrame by `Company_ID` and then calculate the total number of transactions, total quantity purchased, total spending, and average transaction value for each company using the `agg()` method.



In [ ]:
customer_aggregates = merged_data.groupby('Company_ID').agg(
    total_transactions=('Transaction_ID', 'count'),
    total_quantity_purchased=('Quantity', 'sum'),
    total_spending=('Total_Cost', 'sum'),
    average_transaction_value=('Total_Cost', 'mean')
).reset_index()

print("First few rows of the customer_aggregates DataFrame:")
display(customer_aggregates.head())

## Feature Engineering: Product-Centric Aggregates

### Subtask:
Develop features such as total quantity sold per product, total revenue generated by each product, and average product price.


**Reasoning**:
To develop product-centric features, I will group the `merged_data` DataFrame by 'Product_ID' and calculate the total quantity sold, total revenue, and average product price using the `agg()` method.



In [ ]:
product_aggregates = merged_data.groupby('Product_ID').agg(
    total_quantity_sold=('Quantity', 'sum'),
    total_revenue=('Total_Cost', 'sum'),
    average_product_price=('Product_Price_x', 'mean')
).reset_index()

print("First few rows of the product_aggregates DataFrame:")
display(product_aggregates.head())

## Outlier Treatment (Conditional)

### Subtask:
Based on the observed distributions and potential outliers in numerical columns ('Transaction_ID', 'Product_Price_x', 'Total_Cost', 'Company_Profit'), apply appropriate outlier handling techniques if necessary for model robustness.


## Summary:

### Q&A
*   **Was 'Transaction_Date' successfully converted to a datetime format?**
    Yes, the 'Transaction_Date' column was successfully converted to `datetime64[ns]` format, enabling time-series analysis.
*   **Were missing values in 'Company_Profit', 'Company_Name', 'Address', and 'Product_Name' addressed?**
    Yes, missing values in these columns were successfully addressed using appropriate imputation strategies: median for 'Company\_Profit', and 'Unknown' or 'Unknown Product' for the categorical columns ('Company\_Name', 'Address', 'Product\_Name'). All these columns now have zero missing values.
*   **Were entries where 'Quantity' is 0 investigated?**
    Yes, it was identified that 88 records have a 'Quantity' of 0. These records still contain non-zero values in other financial columns, suggesting they might represent specific transaction types like cancelled orders or returns.

### Data Analysis Key Findings
*   The 'Transaction\_Date' column was successfully converted to a `datetime64[ns]` data type.
*   Before imputation, 'Company\_Profit' had 1148 missing values, 'Company\_Name' and 'Address' each had 536 missing values, and 'Product\_Name' had 530 missing values.
*   Missing values in 'Company\_Profit' were imputed with the median. Missing values in 'Company\_Name' and 'Address' were imputed with 'Unknown', while 'Product\_Name' was imputed with 'Unknown Product'. After imputation, all targeted columns had 0 missing values.
*   A total of 88 records were found where 'Quantity' was 0. These records retained financial values in 'Product\_Price\_x', 'Total\_Cost', and 'Company\_Profit', indicating they are not simply empty entries.
*   Four new time-based features were successfully extracted from 'Transaction\_Date': 'Transaction\_Year', 'Transaction\_Month', 'Transaction\_DayOfWeek', and 'Transaction\_Quarter'.
*   Customer-centric aggregate features were created for each unique `Company_ID`: `total_transactions`, `total_quantity_purchased`, `total_spending`, and `average_transaction_value`. For instance, `Company_ID` 1.0 shows 33 transactions, a total quantity of 347.0, total spending of \$45,029,600.0, and an average transaction value of approximately \$1.36 million.
*   Product-centric aggregate features were developed for each unique `Product_ID`: `total_quantity_sold`, `total_revenue`, and `average_product_price`. For example, `Product_ID` 1.0 has a `total_quantity_sold` of 2443.0, a `total_revenue` of \$344,680,000.0, and an `average_product_price` of \$139,805.89.

### Insights or Next Steps
*   The 'Quantity' = 0 records warrant further investigation to determine if they should be filtered out for sales analysis or if they represent important business events like returns or order cancellations, requiring specific handling (e.g., negative quantities for returns).
*   The newly engineered time-based, customer-centric, and product-centric features provide a rich foundation for advanced analysis, such as trend forecasting, customer segmentation, and product performance evaluation. These features should be merged back into the main DataFrame for subsequent modeling.


# Draft of Milestone 1: Project Predictive Model



## Prepare Features for Modeling

Merge the customer-centric and product-centric aggregate features into the `merged_data` DataFrame, and then prepare all features for modeling by identifying and segregating numerical and categorical features for subsequent processing.

### Subtask:
Selecting the relevant features from the `merged_data` DataFrame and the newly created aggregate DataFrames (`customer_aggregates`, `product_aggregates`).




**Reasoning**:
To select relevant features, I will first display the column names of the `merged_data` DataFrame to understand its current structure, which includes transaction-level and time-based features.



In [ ]:
print("Columns in merged_data DataFrame:")
display(merged_data.columns)

**Reasoning**:
To identify the customer-centric features available for merging, I will display the column names of the `customer_aggregates` DataFrame.



In [ ]:
print("Columns in customer_aggregates DataFrame:")
display(customer_aggregates.columns)

**Reasoning**:
To identify the product-centric features available for merging, I will display the column names of the `product_aggregates` DataFrame.



In [ ]:
print("Columns in product_aggregates DataFrame:")
display(product_aggregates.columns)

#### Merging Keys Identified

*   **Customer-centric features** from `customer_aggregates` will be merged with `merged_data` using the **`Company_ID`** column.
*   **Product-centric features** from `product_aggregates` will be merged with `merged_data` using the **`Product_ID`** column.

**Reasoning**:
Now that the merging keys are identified, I will proceed to merge the customer-centric and product-centric aggregate features into the `merged_data` DataFrame. Merging customer aggregates first, then product aggregates.



In [ ]:
merged_data = pd.merge(merged_data, customer_aggregates, on='Company_ID', how='left')
merged_data = pd.merge(merged_data, product_aggregates, on='Product_ID', how='left')

print("First 5 rows of merged_data after adding aggregate features:")
display(merged_data.head())
print("Shape of merged_data after adding aggregate features:")
display(merged_data.shape)

Now that the merged_data DataFrame is complete with all relevant features, I will identify and separate the numerical and categorical columns. This segregation is crucial for preprocessing steps such as scaling numerical features and encoding categorical features, which are necessary before model training.



In [ ]:
numerical_features = merged_data.select_dtypes(include=np.number).columns.tolist()
categorical_features = merged_data.select_dtypes(include='object').columns.tolist()

print("Numerical Features:")
display(numerical_features)
print("\nCategorical Features:")
display(categorical_features)

## Encode Categorical Features

### Subtask:
Apply appropriate encoding techniques (e.g., One-Hot Encoding or Label Encoding) to categorical features such as 'Company_Name', 'Address', and 'Product_Name', and temporal features like 'Transaction_Month', 'Transaction_DayOfWeek', and 'Transaction_Quarter', if treated as categorical.


**Reasoning**:
To encode the specified categorical features ('Company_Name', 'Address', 'Product_Name', 'Transaction_Month', 'Transaction_DayOfWeek', 'Transaction_Quarter') using one-hot encoding, I will first ensure the temporal features are treated as categorical, then apply `pd.get_dummies()`, concatenate the result with the original DataFrame, and finally drop the original columns.



In [ ]:
categorical_cols_to_encode = ['Company_Name', 'Address', 'Product_Name', 'Transaction_Month', 'Transaction_DayOfWeek', 'Transaction_Quarter']

# Convert temporal features to object type to treat them as categorical for one-hot encoding
for col in ['Transaction_Month', 'Transaction_DayOfWeek', 'Transaction_Quarter']:
    merged_data[col] = merged_data[col].astype('object')

# Apply one-hot encoding
encoded_features = pd.get_dummies(merged_data[categorical_cols_to_encode], drop_first=False)

# Concatenate the encoded features with the original DataFrame
merged_data = pd.concat([merged_data, encoded_features], axis=1)

# Drop the original categorical columns
merged_data.drop(columns=categorical_cols_to_encode, inplace=True)

print("First 5 rows of merged_data after one-hot encoding:")
display(merged_data.head())
print("Shape of merged_data after one-hot encoding:")
display(merged_data.shape)

## Handle Numerical Features

### Subtask:
Ensure numerical features are suitable for modeling, potentially involving scaling or standardization.


**Reasoning**:
To ensure numerical features are suitable for modeling, I will first identify all numerical columns in the `merged_data` DataFrame after one-hot encoding, as the one-hot encoding process adds many new numerical columns.



In [ ]:
import numpy as np

current_numerical_features = merged_data.select_dtypes(include=np.number).columns.tolist()

print("Numerical Features after one-hot encoding:")
display(current_numerical_features)

**Reasoning**:
To standardize the numerical features, I will import `StandardScaler`, instantiate it, apply `fit_transform` to the `current_numerical_features`, and update the DataFrame. Finally, I will display descriptive statistics to confirm the scaling.



In [ ]:
from sklearn.preprocessing import StandardScaler

# Instantiate the StandardScaler
scaler = StandardScaler()

# Apply fit_transform to the numerical features and update the DataFrame
merged_data[current_numerical_features] = scaler.fit_transform(merged_data[current_numerical_features])

print("Descriptive statistics for numerical columns after scaling:")
display(merged_data[current_numerical_features].describe())

## Define Target Variable

### Subtask:
Determine the target variable(s) for learning purchasing behavior. This could be future 'Quantity', 'Total_Cost', or a derived metric like customer churn, next purchase likelihood, or product recommendation.


## Summary:

### Data Analysis Key Findings

*   **Feature Merging and Preparation**:
    *   Customer-centric aggregate features (e.g., `total_transactions`, `total_spending`) from `customer_aggregates` were successfully merged into the main dataset using `Company_ID`.
    *   Product-centric aggregate features (e.g., `total_quantity_sold`, `total_revenue`) from `product_aggregates` were successfully merged using `Product_ID`.
    *   After merging, the `merged_data` DataFrame expanded to 22 columns and contained 5329 rows.
    *   Numerical features identified included `Transaction_ID`, `Company_ID`, `Product_ID`, `Quantity`, `Product_Price_x`, `Total_Cost`, `Company_Profit`, temporal features, and all aggregate features.
    *   Categorical features identified were `Company_Name`, `Address`, and `Product_Name`.
*   **Categorical Feature Encoding**:
    *   The features `Company_Name`, `Address`, `Product_Name`, `Transaction_Month`, `Transaction_DayOfWeek`, and `Transaction_Quarter` were selected for encoding.
    *   Temporal features (`Transaction_Month`, `Transaction_DayOfWeek`, `Transaction_Quarter`) were explicitly converted to `object` type to be treated as categorical.
    *   One-hot encoding was applied to these features, resulting in a significant expansion of the DataFrame. The `merged_data` DataFrame's shape changed from 5329 rows and 22 columns to 5329 rows and 237 columns, indicating the creation of 215 new binary columns.
    *   The original categorical columns were dropped after encoding.
*   **Numerical Feature Scaling**:
    *   All identified numerical features were scaled using `StandardScaler`.
    *   Post-scaling descriptive statistics confirmed that the numerical features now have a mean very close to 0 and a standard deviation very close to 1, indicating successful standardization.

### Insights or Next Steps

*   The dataset is now fully prepared with both categorical and numerical features transformed appropriately for machine learning model training.
*   The next logical step is to proceed with model selection, training, and evaluation using this processed dataset.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

customers = pd.read_csv("customer_data.csv")
products = pd.read_csv("products_data.csv")
transactions = pd.read_csv("transactions_data.csv")

print(customers.shape)
print(products.shape)
print(transactions.shape)

(100, 15)
(20, 4)
(10000, 9)


In [ ]:
# Re-load customers and products with the correct separator
customers = pd.read_csv("customer_data.csv", sep=None, engine="python")
products  = pd.read_csv("products_data.csv", sep=None, engine="python")

# Clean column names again
customers.columns = customers.columns.str.strip().str.lower().str.replace(" ", "_")
products.columns  = products.columns.str.strip().str.lower().str.replace(" ", "_")

print("customers cols:", list(customers.columns))
print("products cols:", list(products.columns))


customers cols: ['\ufeff', 'company_id', 'total_orders', 'total_quantity', 'total_spent', 'avg_order_value', 'unique_products', 'last_purchase', 'first_purchase', 'qty_outlier_count', 'cost_outlier_count', 'recency_days', 'customer_age_days', 'avg_qty_per_order', 'will_buy_next_30d']
products cols: ['\ufeff', 'product_id', 'product_name', 'product_price']


In [ ]:
# Remove BOM artifact column if it exists
if '\ufeff' in customers.columns:
    customers = customers.drop(columns=['\ufeff'])

if '\ufeff' in products.columns:
    products = products.drop(columns=['\ufeff'])

print("customers cols (fixed):", list(customers.columns))
print("products cols (fixed):", list(products.columns))


customers cols (fixed): ['company_id', 'total_orders', 'total_quantity', 'total_spent', 'avg_order_value', 'unique_products', 'last_purchase', 'first_purchase', 'qty_outlier_count', 'cost_outlier_count', 'recency_days', 'customer_age_days', 'avg_qty_per_order', 'will_buy_next_30d']
products cols (fixed): ['product_id', 'product_name', 'product_price']


In [ ]:
merged_data = pd.merge(transactions, customers, on="company_id", how="left")
merged_data = pd.merge(merged_data, products, on="product_id", how="left")

print("Merged shape:", merged_data.shape)


Merged shape: (10000, 24)


In [ ]:
# Define target variable
TARGET_COL = "high_value_transaction"


In [ ]:
# Drop leakage and non-feature columns
drop_cols = ["transaction_id", "transaction_date", "total_cost", "unnamed:_0"]
drop_cols = [c for c in drop_cols if c in merged_data.columns]

X = merged_data.drop(columns=[TARGET_COL] + drop_cols)
y = merged_data[TARGET_COL]

# Keep numeric features only
X = X.select_dtypes(include=[np.number])

# Handle missing values from merge
X = X.fillna(0)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (10000, 16)
y shape: (10000,)


For baseline modeling we restrict to numeric features only; future work will encode categorical features.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


NameError: name 'X' is not defined

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

print("✅ Model trained: Logistic Regression")


✅ Model trained: Logistic Regression


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred))
print("\nClassification Report:\n", classification_report(y_test, pred))


Accuracy: 0.89

Confusion Matrix:
 [[1399   95]
 [ 125  381]]

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      1494
           1       0.80      0.75      0.78       506

    accuracy                           0.89      2000
   macro avg       0.86      0.84      0.85      2000
weighted avg       0.89      0.89      0.89      2000

